In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class Encoder(nn.Module):
    def __init__(self, vocab_size, block_size=256, embed_dim=64, num_layers=4):
        super().__init__()
        self.block_size = block_size
        self.transformer = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.Sequential(*[Layer(embed_dim, block_size) for _ in range(num_layers)]),
            nn.LayerNorm(embed_dim),
        )
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, x):
        return self.lm_head(self.transformer(x))

class Layer(nn.Module):
    def __init__(self, embed_dim, block_size):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.qkv = nn.Linear(embed_dim, embed_dim * 3, bias=False)
        self.proj = nn.Linear(embed_dim, embed_dim, bias=False)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(approximate="tanh"),
            nn.Linear(4 * embed_dim, embed_dim),
        )
        t = torch.arange(-block_size+1, 1)
        self.register_buffer("alibi", torch.tril(torch.cat([t, t[:-1]]).unfold(0, len(t), 1).flip(0)))

    def forward(self, x):
        B, T, C = x.shape
        q, k, v = self.qkv(self.ln1(x)).chunk(3, dim=-1)
        w = q @ k.transpose(-2, -1) / math.sqrt(C) # (B, T, T)
        w = w + self.alibi[:T, :T]
        w = F.softmax(w, dim=-1)
        attn = self.proj(w @ v) # (B, T, C)

        x = x + attn
        x = x + self.mlp(self.ln2(x))
        return x

In [2]:
import lightning as pl
from shared import corpus, tokenizers, trainers

text = corpus.shakespeare()
tokenizer = tokenizers.unique_chars(text, mask_token="😷")

pl.seed_everything(89026614)
model = Encoder(tokenizer.get_vocab_size())
trainer = trainers.MLMTrainer(model, tokenizer, device = "mps")
trainer.train(text, batch_size=36, epochs=25)

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset tiny_shakespeare (/Users/cztomsik/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 443.51it/s]
Global seed set to 89026614
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in 

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 23.88it/s]

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Hello, my n😷me is
[[('J', 0.06201637163758278), ('F', 0.033590883016586304), (' ', 0.03334825858473778)]]
                                                                           

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 202/202 [00:07<00:00, 26.36it/s, loss=3.27, v_num=48]Hello, my n😷me is
[[(' ', 0.17127932608127594), ('e', 0.09036801010370255), ('t', 0.058753374963998795)]]
Epoch 1: 100%|██████████| 202/202 [00:07<00:00, 27.95it/s, loss=3.19, v_num=48, test_loss=3.280]Hello, my n😷me is
[[('e', 0.11139622330665588), (' ', 0.10040083527565002), ('t', 0.07105366885662079)]]
Epoch 2: 100%|██████████| 202/202 [00:07<00:00, 27.98it/s, loss=3.1, v_num=48, test_loss=3.220] Hello, my n😷me is
[[(' ', 0.11714823544025421), ('e', 0.08352139592170715), ('h', 0.07656692713499069)]]
Epoch 3: 100%|██████████| 202/202 [00:07<00:00, 28.03it/s, loss=3, v_num=48, test_loss=3.100]   Hello, my n😷me is
[[(' ', 0.21999499201774597), ('e', 0.09472627937793732), ('a', 0.07238544523715973)]]
Epoch 4: 100%|██████████| 202/202 [00:07<00:00, 27.79it/s, loss=2.82, v_num=48, test_loss=2.980]Hello, my n😷me is
[[(' ', 0.35349783301353455), ('e', 0.11356089264154434), ('a', 0.10928848385810852)]]
Epoch 5: 10

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 202/202 [00:11<00:00, 17.55it/s, loss=1.84, v_num=48, test_loss=1.790]


In [3]:
trainer.wrapper.fill("Make n😷 more")

[[('o', 0.35112264752388),
  ('a', 0.27821195125579834),
  ('i', 0.21505515277385712)]]